# MNIST

## Instalación

Descargamos la imagen

`docker pull ermaker/keras-jupyter`

Ejecutamos y vamos al navegador en la dirección `0.0.0.0:8888`

`docker run -d -p 8888:8888 \ --name aml-deep-learning \ 
-v /home/nanounanue/proyectos/advanced-machine-learning:/notebook \ 
ermaker/keras.jupyter`

## Bibliotecas utilizadas

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

## Descargando y preparando los datos

In [ ]:
batch_size = 128
nb_classes = 10

img_rows = 28
img_cols = 28


(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

X_train /= 255
X_test /= 255

print 'X_train shape: {}'.format(X_train.shape)
print '{} muestras de entranamiento'.format(X_train.shape[0])
print '{} muestras de prueba'.format(X_test.shape[0])

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print 'One hot encoding: {}'.format(Y_train[0, :])

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_train[i,0], cmap='gray')
    plt.axis('off')

## Generando el modelo

Este modelo que vamos a construir esta basado en el tutorial: 

`http://eblearn.sourceforge.net/beginner_tutorial2_train.html`

<img src="lenet5.png">

En el se puede leer la descripción (esto tendrá más sentido conforme avancemos en la clase):

```
Let us first list out what we already know about how we want our convnet to be.

    The input images are 32×32 in size with 1 channel(i.e grayscale image, not color)
    From figure 1, we can say that there are 6-layers in our convnet.
        Layer C1 is a convolution layer with 6 feature maps and a 5×5 kernel for each feature map.
        Layer S1 is a subsampling layer with 6 feature maps and a 2×2 kernel for each feature map.
        Layer C3 is a convolution layer with 16 feature maps and a 6×6 kernel for each feature map.
        Layer S4 is a subsampling layer with 16 feature maps and a 2×2 kernel for each feature map.
        Layer C5 is a convolution layer with 120 feature maps and a 6×6 kernel for each feature map.
        Layer C6 is a fully connected layer
    Let us use an additive bias and a tanh non-linearity for each layer(several other non-linearity operations can be used, like sigmoid, softmax etc.). If you do not understand why we are doing this, please look at the ConvNets section
    Let us assuming full connections between each layer, i.e all feature maps in layer X are connected with all feature maps in layer X+1. In the next tutorial, you will see how to define more complicated connections.

```

In [ ]:
model = Sequential()

model.add(Convolution2D(6, 5, 5, input_shape=(1, img_rows, img_cols), border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16, 5, 5, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(120, 5, 5))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

# Entrenamiento

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=["accuracy"])  

In [ ]:
nb_epoch = 2

model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = nb_epoch,
         verbose = 1, validation_data=(X_test, Y_test))

In [ ]:
score = model.evaluate(X_test, Y_test, show_accuracy = True, verbose = 0)

In [ ]:
print 'Test score: {}'.format(score[0])
print 'Test accuracy: {}'.format(score[1])

# Visualizando resultados

In [ ]:
res = model.predict_classes(X_test[:30])

plt.figure(figsize = (20, 20))

for i in range(30):
    plt.subplot(5, 6, i+1)
    plt.imshow(X_test[i,0], cmap='gray')
    plt.gca().get_xaxis().set_ticks([])
    plt.gca().get_yaxis().set_ticks([])
    plt.ylabel("Prediccion = %d" % res[i], fontsize=16)